# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration. You are allowed to use the FPHF features and have to implement the RANSAC algorithm

2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [1]:
# from ex1 

import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size, source, target):

    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registrations(source, target)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh


# Start tasks

## Task 1.1

Implementation of RANSAC


1. find pairs in fpfh (source <-> target)
2. take N number of pairs and run Kabsch -> rot, trans = tranformation
3. applying transformation
4. calc distance of pairs and sum = error
5. find best model

 
### PSEUDOCODE
 
 ```
 RANSAC(k=maxinter, n= numberofrandompoints, source_fpfh, target)
 
 for ALL source
     onesource
     for all target
         onetarget
         dist = norm(1sou-1tar)
 
 vector_sorted_source
 vector _ target
 
 while iter < k:
 
 
     find n pairs
 
     points = rand n
     
     rotmat,transvec = kabsch(points)
     
     target_transformed = rotmat*target + transvec
     
     error = norm2( source, target)
     
     if error < bestErr then
            bestFit := r,t
            bestErr := thisErr
     end if
     
     return bestFit
 
 ```
        





In [2]:
r1 = o3d.io.read_point_cloud("global_registration/r1.pcd")
r2 = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, r1, r2)



:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [3]:
print(source_fpfh)

Feature class with dimension = 33 and num = 4760
Access its data via data member.


In [4]:
s = source_fpfh.data
t = target_fpfh.data

dist_t = np.zeros([target_fpfh.num(),1])
index = []


for u in range(50):
        ps = s[:,u]
        
        for i in range(target_fpfh.num()):
            pt = t[:,i]
            dist_t[i] = np.linalg.norm(ps-pt)
        
        if u>= target_fpfh.num():
            break
        index.append(np.argmin(dist_t))
        
            
pairs_target_points = np.asarray(target_down.points)[index,:]
# print("Done")  
# print("pairs",pairs_target_points)
# print("index",index)
# print("target_down",target_down.points[0])

In [5]:
def Kabsch(A, B):
    assert len(A) == len(B)
    
    
    N = A.shape[0];  # total points

    
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)

    # center the points
    AA = A - centroid_A
    BB = B - centroid_B
    
    H = np.transpose(BB) * AA
    
    print(H.shape)

    U, S, Vt = np.linalg.svd(H)

    R = np.matmul(U,Vt)

    t = centroid_B - np.matmul(R, centroid_A)
    
    return R, t


In [6]:
max_iterations = 1
import random
import matplotlib.pyplot as plt
my_source = np.asarray(source_down.points)
my_target = np.asarray(target_down.points)
my_index = np.asarray(index)
while max_iterations:
            max_iterations -= 1
            # Add 3 random indexes
            random.seed()
            inliers = []
            n = 3
            while len(inliers) < n:
                random_index = random.randint(0, 50-1)
                #random_index = random.randint(0, len(s[0,:])-1)
                inliers.append(random_index)
            match_source = my_source[inliers]
            my_target_index = my_index[inliers]
            match_target = my_target[my_target_index]
            print("match target",match_target)
            print(" ")
            print("match source",match_source)
            
            
            R,t = Kabsch(match_source,match_target)
            
            print('Kabsch result:')
            print('R:',R)
            print('t',t)
            
            t.resize(3,1)
            
            target_tranformed = np.dot(R,my_target.T) + t
            
            calc error
            
            find best model
            
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(my_source[:,0], my_source[:,1],my_source[:,2])
            ax.scatter
            
            
            
#             error = 

SyntaxError: invalid syntax (<ipython-input-6-35c949503ae3>, line 35)